In [ ]:
import pandas as pd
import requests
from io import BytesIO
import re

In [ ]:
# Defnining a function to download and combine excel files into a dataframe. 

# Takes as input a list of urls, and a number of rows to skip, which varies by year.

def download_excel_files(urls, rows_to_skip, sheet_number = 1):
    all_data = []

    for i, url in enumerate(urls, 1):
        # Download an Excel file
        response = requests.get(url)
        
        if response.status_code == 200:
            # Read the Excel file into a BytesIO object
            excel_file = BytesIO(response.content)
            
            # Read the second sheet of the Excel file, skip the first 4 rows of metadata
            df = pd.read_excel(excel_file, skiprows = rows_to_skip, sheet_name = sheet_number)
            
            # Add columns to identify which file this data came from
            df['source_file'] = f"file_{i}"
            df['year'] = url.split('/')[-1].split('_')[0][-2:]  # Extract year from filename
            
            all_data.append(df)
            
            print(f"Downloaded and processed file {i}")
        else:
            print(f"Failed to download file from {url}. Status code: {response.status_code}")

    if all_data:
        # Combine all DataFrames into one
        combined_df = pd.concat(all_data, ignore_index=True)
        return combined_df
    else:
        print("No data was successfully downloaded.")
        return None

In [ ]:
# Attempted to use beautifulsoup to scrape the specialty care clinic file urls, but neither xpath or css selector worked reliably. 

# I split the urls into 2 lists, pre_2018_urls and post_2018_urls, on account of inconsistent data structure.

# I also define a data dictionary url to create a dataframe that can be used to standardize column names between the newer and older datasets. 

# The plan is to create two dataframes for pre- and post-2018, modify the former's structure to match the latter, and combine.

# Define a list of urls for files pertaining to 2013–2017

pre_2018_urls = [
    "https://data.chhs.ca.gov/dataset/17bbc0b0-869e-4168-b03b-48fa60c78577/resource/896c699c-07fc-4049-bda0-ff98ac8e3913/download/spcl13utildatafinal.xlsx",
    "https://data.chhs.ca.gov/dataset/17bbc0b0-869e-4168-b03b-48fa60c78577/resource/91fa31b7-8f40-47f1-8bca-bbc063221993/download/spcl14utildatafinal.xlsx",
    "https://data.chhs.ca.gov/dataset/17bbc0b0-869e-4168-b03b-48fa60c78577/resource/171f7631-4cb2-4b20-b238-d5ab3512ae10/download/spcl15utildatafinal.xlsx",
    "https://data.chhs.ca.gov/dataset/17bbc0b0-869e-4168-b03b-48fa60c78577/resource/c6a99713-427a-44df-947d-d46c3402a4d6/download/spcl16_util_data_final-ver2.xlsx",
    "https://data.chhs.ca.gov/dataset/17bbc0b0-869e-4168-b03b-48fa60c78577/resource/e7a2def1-c0dd-41af-a283-46e095bc0af2/download/spcl17_util_data_final.xlsx"
]

# Define a list of urls for files pertaining to 2018–2023

post_2018_urls = [
    "https://data.chhs.ca.gov/dataset/17bbc0b0-869e-4168-b03b-48fa60c78577/resource/8ad9b464-cbbd-4ad5-b37d-d2daa924768b/download/spcl23_util_data_prelim.xlsx",
    "https://data.chhs.ca.gov/dataset/17bbc0b0-869e-4168-b03b-48fa60c78577/resource/00a9d637-d75a-4ba5-9ed5-87bb01f3a6e3/download/spcl22_util_data_final.xlsx",
    "https://data.chhs.ca.gov/dataset/17bbc0b0-869e-4168-b03b-48fa60c78577/resource/f6339c46-8e35-4466-b972-ce132c43cbf4/download/spcl21_util_data_final.xlsx",
    "https://data.chhs.ca.gov/dataset/17bbc0b0-869e-4168-b03b-48fa60c78577/resource/9c883633-b661-4da3-b39f-50536f60e573/download/spcl20_util_data_final.xlsx",
    "https://data.chhs.ca.gov/dataset/17bbc0b0-869e-4168-b03b-48fa60c78577/resource/188b31e3-2307-479e-9bee-632083f902ba/download/spcl19_util_data_final.xlsx",
    "https://data.chhs.ca.gov/dataset/17bbc0b0-869e-4168-b03b-48fa60c78577/resource/e891cdff-6092-4316-b406-dcbcf4a9c016/download/spcl18_util_data_final.xlsx"
]

data_dictionary_url = ["https://data.chhs.ca.gov/dataset/17bbc0b0-869e-4168-b03b-48fa60c78577/resource/188b31e3-2307-479e-9bee-632083f902ba/download/spcl19_util_data_final.xlsx"]

In [ ]:
pre_2018_df = download_excel_files(pre_2018_urls, rows_to_skip = [1,2,3])

post_2018_df = download_excel_files(post_2018_urls, rows_to_skip = [1,2,3,4])

data_dictionary = download_excel_files(data_dictionary_url, rows_to_skip = 0, sheet_number = 3)

# Data cleaning to merge the two sets of historical data

In [ ]:
pre_2018_df

In [ ]:
post_2018_df

In [ ]:
data_dictionary

In [ ]:
# Creating a dictionary of old an new column names to rename columns in the pre-2018 dataframe.

old_names = data_dictionary["ALIRTS Dataset Header (2017)"]

new_names = data_dictionary["SIERA Dataset Header (2019)"]

name_mapping = dict(zip(old_names, new_names))

# Renaming the columns in the pre-2018 dataframe.

pre_2018_df = pre_2018_df.rename(columns=name_mapping)

In [ ]:
pre_2018_df

In [ ]:
# Creating a function to combine street address columns in pre-2018 dataframe.

def combine_street_address(df, col1, col2, new_col_name):
    
    # Combine columns, handling NaN values
    df[new_col_name] = df[col1].fillna('').astype(str) + df[col2].fillna('').apply(lambda x: f', {x}' if x else '')
    
    # Remove trailing comma and space if col2 was empty
    df[new_col_name] = df[new_col_name].str.rstrip(', ')

    # Remove original columns
    df.drop(columns=[col1, col2], inplace=True)

    return df

In [ ]:
# Creating combined street adress and parent company address columns in the pre-2018 dataframe.

combine_street_address(pre_2018_df, "FAC_ADDRESS_ONE", "FAC_ADDRESS_TWO", "FAC_STR_ADDR")

combine_street_address(pre_2018_df, "PARENT_ADDRESS_ONE", "PARENT_ADDRESS_TWO", "FAC_PAR_CORP_BUS_ADDR")

In [ ]:
post_2018_df

In [ ]:
def compare_columns(df1, df2):
    set1 = set(df1.columns)
    set2 = set(df2.columns)
    
    only_in_df1 = set1 - set2
    only_in_df2 = set2 - set1
    
    return only_in_df1, only_in_df2

# Usage
columns_only_in_df1, columns_only_in_df2 = compare_columns(pre_2018_df, post_2018_df)

print("Columns only in df1:", columns_only_in_df1)
print("Columns only in df2:", columns_only_in_df2)

In [ ]:
pre_2018_df["FAC_NO"].nunique()

In [ ]:
post_2018_df["FAC_NO"].nunique()

In [ ]:
def compare_unique_values(df1, df2, column_name):
    # Get unique values from each DataFrame
    unique_df1 = set(df1[column_name].unique())
    unique_df2 = set(df2[column_name].unique())
    
    # Find values in df1 but not in df2
    only_in_df1 = unique_df1 - unique_df2
    
    # Find values in df2 but not in df1
    only_in_df2 = unique_df2 - unique_df1
    
    # Find values in both
    in_both = unique_df1.intersection(unique_df2)
    
    return only_in_df1, only_in_df2, in_both

# Usage
column_to_compare = 'FAC_NO'
only_in_df1, only_in_df2, in_both = compare_unique_values(pre_2018_df, post_2018_df, column_to_compare)

print(f"Unique values only in df1: {only_in_df1}")
print(f"Unique values only in df2: {only_in_df2}")
print(f"Unique values in both: {in_both}")

In [ ]:
import pandas as pd

def compare_shared_column_types(df1, df2):
    # Find shared columns
    shared_columns = list(set(df1.columns) & set(df2.columns))
    
    if not shared_columns:
        print("No shared columns found between the DataFrames")
        return None
    
    # Compare data types
    comparison = {}
    for col in shared_columns:
        type1 = df1[col].dtype
        type2 = df2[col].dtype
        comparison[col] = {
            'df1_type': type1,
            'df2_type': type2,
            'match': type1 == type2
        }
    
    # Convert to DataFrame for easy viewing
    comparison_df = pd.DataFrame.from_dict(comparison, orient='index')
    
    return comparison_df

# Usage
type_comparison = compare_shared_column_types(pre_2018_df, post_2018_df)

if type_comparison is not None:
    print("Shared column type comparison:")
    print(type_comparison)
    
    # Identify mismatched columns
    mismatched = type_comparison[type_comparison['match'] == False]
    if not mismatched.empty:
        print("\nColumns with mismatched types:")
        print(mismatched)
    else:
        print("\nAll shared columns have matching types.")

In [ ]:
# Converting aquisition-related columns in pre-2018 dataframe to types in post-2018 dataframe.

# Pre-2018 dataframe doesn't contain any values for these columns pre-2018, which is why they were imputed differently from those post-2018, which do contain information.

pre_2018_df["DT_ACQUIRE_01"] = pd.to_datetime(pre_2018_df["DT_ACQUIRE_01"])

pre_2018_df["DEEQUIP_01"] = pre_2018_df["DEEQUIP_01"].astype("str")

pre_2018_df["MEANS_FOR_ACQUISITION_01"] = pre_2018_df["MEANS_FOR_ACQUISITION_01"].astype("str")

In [ ]:
# Creating a function to remove the string "District " from SENATE_DIST, ASSEMBLY_DIST, and CONGRESS_DIST columns in post-2018 dataframe. 

def clean_and_convert_to_numeric(df, columns):
    def clean_numeric(value):
        if pd.isna(value):
            return value
        # Remove all non-digit characters
        cleaned = re.sub(r'\D', '', str(value))
        return cleaned if cleaned else None

    for col in columns:
        if col not in df.columns:
            print(f"Warning: Column '{col}' not found in the DataFrame. Skipping.")
            continue
        
        # Apply the cleaning function and convert to integer
        df[col] = df[col].apply(clean_numeric).astype('Int64')
    
    return df

columns_to_clean = ['SENATE_DIST', 'CONGRESS_DIST', 'ASSEMBLY_DIST']
post_2018_df = clean_and_convert_to_numeric(post_2018_df, columns_to_clean)

In [ ]:
# Creating a function to merge the pre-2018 and post-2018 dataframes
# Finds all shared columns between df1 and df2 using set intersection.
# Checks if there are any shared columns. If not, it raises an error.
# Merges the DataFrames using all shared columns.

# Using how='outer' ensures that all rows from both DataFrames are kept, even if there's no match on all shared columns.
# suffixes=('_df1', '_df2') are added to disambiguate column names that are in both DataFrames but weren't used for merging.

def merge_on_shared_columns(df1, df2):
    # Find shared columns
    shared_columns = list(set(df1.columns) & set(df2.columns))
    
    # Ensure there are shared columns
    if not shared_columns:
        raise ValueError("No shared columns found between the DataFrames")
    
    # Merge DataFrames on all shared columns
    merged_df = pd.merge(df1, df2, on=shared_columns, how='outer', suffixes=('_df1', '_df2'))
    
    return merged_df

In [ ]:
try:
    merged_df = merge_on_shared_columns(pre_2018_df, post_2018_df)
    print("Merge successful")
    print(f"Shared columns used for merging: {shared_columns}")
    print(f"Shape of merged DataFrame: {merged_df.shape}")
except ValueError as e:
    print(f"Error: {e}")

# Checking for duplicate rows:

pythonCopyduplicate_rows = merged_df[merged_df.duplicated()]

if not duplicate_rows.empty:
    print(f"Number of duplicate rows: {len(duplicate_rows)}")

# For null values created by the merge:

pythonCopynull_counts = merged_df.isnull().sum()
print("Null value counts per column:")
print(null_counts[null_counts > 0])

# Verify the merge result:

pythonCopyprint(f"Rows in df1: {len(df1)}")
print(f"Rows in df2: {len(df2)}")
print(f"Rows in merged_df: {len(merged_df)}")
print(f"Columns in merged_df: {len(merged_df.columns)}")

In [ ]:
print(merged_df["FAC_NO"].nunique())
print(post_2018_df["year"].nunique())

In [ ]:
merged_df


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9a5367f9-570e-4fa7-91cf-2de38c70a230' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>